In [ ]:
import pandas as pd
import numpy as np
import os
import importlib
import random
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, average_precision_score
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForMaskedLM, BertConfig, BertModel, InputExample
from run_classifier_dataset_utils import convert_examples_to_features, parallel_convert_examples_to_features
from tqdm.auto import tqdm
from torch.utils import data
from scipy import stats
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', 50)
pd.options.mode.chained_assignment = None

## Preliminaries

Run the `create_data_bigquery.ipynb` script to generate the following files:

- `cohort.h5`:
  - Contains one record for each adult patient’s first ICU stay over 48 hours in lengthwithin their first hospital admission.
  - The `mort_icu` column represents whether the patient died during their ICU stay.
  - The columns from `Acute Renal` to `Shock` correspond to each of the 25 CCS code  groups, which are derived from  ICD-9 codes assigned at the end of a patient’s hospital stay.
  - The `Any Acute` and `Any Chronic` columns are derived from whether the patient has any acute and chronic phenotypes respectively.
  
  
- `notes.h5`
  - Contains, for each of the patients in `cohort.h5`, all of the notes written during their hospital stay (along with the timestamp) for the following note types:
      - Discharge Summary
      - Nursing
      - Nursing/other
  - The notes have been lightly preprocessed (ex: removing PHI identifiers, removing section numbers).

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data_path = '/content/gdrive/My Drive/hst953_hw1/mimic_data'
bert_path = '/content/gdrive/My Drive/hst953_hw1/mimic_data/biobert_pretrain_output_all_notes_150000/'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = BertForMaskedLM.from_pretrained(bert_path).to(device)
tokenizer = BertTokenizer.from_pretrained(bert_path)

In [ ]:
cohort = pd.read_hdf(os.path.join(data_path, 'cohort.h5'))
notes = pd.read_hdf(os.path.join(data_path, 'notes.h5'))

In [ ]:
cohort.head()

In [ ]:
cohort.shape

In [ ]:
notes.head()

In [ ]:
notes.shape

## Sentence Completion

In [ ]:
def fill_blank(text: str, model: BertForMaskedLM, tokenizer: BertTokenizer) -> (str, dict):
    '''
    Given a sentence with a single blank (denoted by an underscore), queries the BERT model to 
        fill in the missing token.
        
    Inputs:
        - text: sentence containing a single underscore corresponding to the missing token
                ex: "[CLS] 40 yo asian homeless man with h/o polysubstance abuse and recently released from _  [SEP]"
        - model: pytorch ClinicalBERT model, of type BertForMaskedLM
        - tokenizer: BertTokenizer object
    
    Output:
        - tuple consisting of the following:
            - string corresponding to the sentence where the underscore is replaced with the most likely token
                ex: "[CLS] 40 yo asian homeless man with h / o polysubstance abuse and recently released from home [SEP]"
            - a dictionary str:float mapping each word in the vocabulary to its normalized probability.
                - sum of the values should be equal to 1
                - the dictionary should have 28996 elements
    '''
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)

    pass

In [ ]:
def test_fill_blank():
    text = '[CLS] 40 yo asian homeless man with h/o polysubstance abuse and recently released from _ [SEP]'
    a,b = fill_blank(text, model, tokenizer)
    assert(a.split(' ')[-2] == 'home'), 'Most likely word not correct!'
    assert(math.isclose(np.sum(list(b.values())), 1.0, rel_tol = 1e-4)), 'Probabilities not normalized!'
    assert(math.isclose(b['shelter'], 0.021500807255506516, rel_tol = 1e-4)), "Probability not correct!"
    print("Test passed!")
    
test_fill_blank()

## Fine-tuning ClinicalBERT

Before starting this section, ensure that you are on a Colab GPU runtime with 25 GB of RAM.

In [ ]:
# reload model to output hidden states
config = BertConfig.from_pretrained(bert_path, output_hidden_states=True)
model = BertModel.from_pretrained(bert_path, config = config).to(device)
tokenizer = BertTokenizer.from_pretrained(bert_path)
n_gpu = torch.cuda.device_count()
if device == 'cuda' and n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [ ]:
notes.category.value_counts()

In [ ]:
print(notes.groupby(['subject_id', 'hadm_id']).agg({'text':'count'})['text'].value_counts().describe())
time_steps = 35 # choose to take most recent 35 notes for each patient

In [ ]:
notes.loc[notes.category == 'Discharge summary', 'charttime'] = notes[notes.category == 'Discharge summary'].iloc[0]['chartdate'] + pd.Timedelta(days = 1) -  pd.Timedelta(seconds = 1)
cohort = cohort.reset_index(drop = True)

def index_notes(x):
    # take most recent time_steps notes
    if len(x) > time_steps:
        return np.concatenate((-1*np.ones(len(x) - time_steps), np.arange(time_steps)))
    else:
        return np.arange(len(x))
    
notes['note_index'] = -1
notes['note_index'] = notes.sort_values(by = ['subject_id','charttime'], ascending = True).groupby('subject_id').transform(index_notes).astype(int)
notes = notes[notes.note_index >= 0] # drop excess notes > time_steps

mapping = dict(map(reversed, cohort['subject_id'].to_dict().items()))
notes.shape

In [ ]:
def convert_input_example(subject_id, note_ind, text):
    return InputExample(guid = '%s-%s'%(subject_id, note_ind), text_a = text, text_b = None, label = 0)

examples = [convert_input_example(row['subject_id'], row['note_index'], row['text']) for idx, row in notes.iterrows()]

In [ ]:
# will take ~(150/n_cpu) min
features = parallel_convert_examples_to_features(examples, 512, tokenizer, 
                                                 output_mode = 'classification', n_cpus = 10)

In [ ]:
class MIMICDataset(data.Dataset):
    def __init__(self, features):
        self.features = features
        self.length = len(features)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        all_input_ids = torch.tensor(self.features[index].input_ids, dtype = torch.long)
        all_input_mask = torch.tensor(self.features[index].input_mask, dtype = torch.long)
        all_segment_ids = torch.tensor(self.features[index].segment_ids, dtype = torch.long)
        y = torch.tensor(self.features[index].label_id, dtype = torch.float32)
        guid = self.features[index].guid

        return all_input_ids, all_input_mask, all_segment_ids, y, guid

def extract_embeddings(v):
    return torch.cat((v[-1][:, 0, :] , v[-2][:, 0, :] , v[-3][:, 0, :] , v[-4][:, 0, :]), 1)

def get_embs(generator):
    model.eval()
    with torch.no_grad():
        for input_ids, input_mask, segment_ids, _,  guid in tqdm(generator):
            input_ids = input_ids.to(device)
            segment_ids = segment_ids.to(device)
            input_mask = input_mask.to(device)
            hidden_states = model(input_ids, token_type_ids = segment_ids, attention_mask = input_mask)[2]
            bert_out = extract_embeddings(hidden_states)
                        
            for c,i in enumerate(guid):
                sub_id, note_idx = i.split('-')
                embs = bert_out[c,:].detach().cpu()
                inputs[mapping[int(sub_id)], int(note_idx), :] = embs
    return True

emb_dim = 768*4

In [ ]:
inputs = torch.zeros((cohort.shape[0], time_steps, emb_dim)) # num_patient x time_steps x emb_size
data_generator = data.DataLoader(MIMICDataset(features), shuffle = True,  batch_size = 64*n_gpu)
# will take several hours
get_embs(data_generator)

In [ ]:
## might want to cache `inputs` to avoid running the above cell again
torch.save(inputs, open('/content/gdrive/My Drive/hst953_hw1/mimic_data/inputs.pt', 'wb'))
# inputs = torch.load(open('/content/gdrive/My Drive/hst953_hw1/mimic_data/inputs.pt', 'rb'))

In [ ]:
seq_lengths = notes.sort_values(by = 'subject_id').groupby('subject_id').agg({'note_index': 'max'})['note_index'].values + 1
targets = pd.read_csv('mapping.csv')['after'].values.tolist()

In [ ]:
cohort.loc[cohort['train'] == 0, 'split'] = 'test'
cohort.loc[(cohort['train'] == 1) & (np.random.rand(len(cohort)) < 0.8), 'split'] = 'train'
cohort['split'] = cohort['split'].fillna('val')

In [ ]:
# Report the following to get the 1 point for the first question of part c
print(inputs[35, 10, 1234])

## Building a Temporal Model
To build your model, you have the following variables defined in memory:
- `targets`: list of 27 predictive targets corresponding to column names in `cohort`
- `cohort`: dataframe containing targets. Make sure to follow the train/val/test split in the `split` column
- `inputs`: num_patient x time_steps (35) x emb_size (3076) tensor. Each [i, :, :] slice of the tensor corresponds to a single patient.
- `mapping`: maps the `subject_id` field to an index in the `inputs` tensor. For example, the features for the patient with subject_id=3 is at the index=0 slice of `inputs`.
- `seq_lengths`: array of size num_patient, where each element represents how many notes (up to 35) the patient had. This is the number of non-zero note embeddings that a patient has in `inputs`.

In [ ]:
# Your code goes here
## train model
## save model as model.pt


In [ ]:
# evaluate on test set
test_cohort = cohort[cohort.split == 'test']
## evaluate your model on test_cohort here
## for each field `i` in targets, write your predictions into a new column called 'pred' + i




In [ ]:
# 5c table
# Run this code and paste the table into your report
aucs = []
for i in targets:
    aucs.append((i, roc_auc_score(test_cohort[i], test_cohort['pred'+i]),
                log_loss(test_cohort[i], test_cohort['pred'+i]),
                average_precision_score(test_cohort[i], test_cohort['pred'+i])))
aucs.append(('Mean', np.mean([i[1] for i in aucs]),np.mean([i[2] for i in aucs]),np.mean([i[3] for i in aucs])))
res = pd.DataFrame(aucs, columns = ['Task','AUROC', 'logloss', "AUPRC"])
res.style.format({
    'AUROC': '{:.3f}',
    'logloss': '{:.3f}',
    'AUPRC': '{:.3f}'
})